# Setup

We use the Penn World Tables and OECD data to produce estimates of $(\mu,\alpha,\delta_k)$ as input in the model (auxiliary parameters). This yields Table 6 in the paper as well as a csv file that collects parameters for the model. 

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os 


In [2]:
countries = ['us','fr','it','dk','sp','nl','se','de']
cnames = ['United States','France','Italy','Denmark','Spain','Netherlands','Sweden','Germany']
co = dict(zip(countries,cnames))
ce = dict(zip(cnames,countries))
data = '../data_sources/'

We load the Penn World Table for some of the stats

In [3]:
years = range(1995,2016,1)
penn = pd.read_excel(data+'pwt/pwt90.xlsx',sheet_name='Data')
penn = penn[(penn['year'].isin(years)) & (penn['country'].isin(cnames))]

FileNotFoundError: [Errno 2] No such file or directory: '../data_sources/pwt/pwt90.xlsx'

We load OECD data for co-insurance rates

In [ ]:
labels = ['country','unit','scrap']
for t in range(1995,2016):
    labels.append(str(t))
oecd = pd.read_excel(data+'oecd/oecd-copays-1995-2015.xls',skiprows=1)
oecd.columns = labels
oecd = oecd.drop(columns=['scrap','unit'])
oecd = oecd.set_index('country')
oecd = oecd.stack().to_frame()
oecd.columns = ['copay']
oecd['copay'] = oecd['copay']/100.0
oecd.index.names = ['country','year']
oecd = oecd.reset_index()

# Co-insurance rates (generosity)

We take from OECD average out-of-pocket health expenditures as a fraction of total. We compute the mean.

In [ ]:
copays = oecd['copay'].groupby(oecd['country']).mean().to_frame()
copays

,copay
country,
Denmark,0.149318
France,0.085540
Germany,0.127620
Italy,0.237656
Netherlands,0.097602
Spain,0.228510
Sweden,0.162742
United States,0.136411


# Depreciation Rate

In [ ]:
delta = penn['delta'].groupby(penn['country']).mean().to_frame()
delta

,delta
country,
Denmark,0.043378
France,0.040203
Germany,0.039408
Italy,0.039708
Netherlands,0.041134
Spain,0.037669
Sweden,0.046767
United States,0.048735


# Labor Share

We take the labor share from Penn World Tables. We compute the mean over the period 1995-2015.

In [ ]:
alpha = penn['labsh'].groupby(penn['country']).mean().to_frame()

In [ ]:
alpha['labsh'] = 1-alpha['labsh']

In [ ]:
alpha.columns = ['alpha']
alpha

,alpha
country,
Denmark,0.360473
France,0.379034
Germany,0.372822
Italy,0.470268
Netherlands,0.393142
Spain,0.373838
Sweden,0.461041
United States,0.383702


# Production of Table 6

This is Table 6 in paper. 

In [ ]:
table = copays

In [ ]:
for x in [alpha,delta]:
    table = table.merge(x,left_index=True,right_index=True)

In [ ]:
table = table[['copay','alpha','delta']]
table.columns = ['mu','alpha','delta']
table = table.transpose()

In [ ]:
for c in table.columns:
    table[c] = table[c].astype('float64')

In [ ]:
table.round(3)

country,Denmark,France,Germany,Italy,Netherlands,Spain,Sweden,United States
mu,0.149,0.086,0.128,0.238,0.098,0.229,0.163,0.136
alpha,0.360,0.379,0.373,0.470,0.393,0.374,0.461,0.384
delta,0.043,0.040,0.039,0.040,0.041,0.038,0.047,0.049


In [ ]:
table.round(3).to_latex('../tables/table_6_auxiliary.tex')

# Saving Auxiliary Parameters

We save the parameters to file fot the model to load. 

In [ ]:
newcolumns = table.columns.to_list()
for i,c in enumerate(table.columns):
    newcolumns[i] = ce[c].upper()
table.columns = newcolumns

In [ ]:
table

,DK,FR,DE,IT,NL,SP,SE,US
mu,0.149318,0.085540,0.127620,0.237656,0.097602,0.228510,0.162742,0.136411
alpha,0.360473,0.379034,0.372822,0.470268,0.393142,0.373838,0.461041,0.383702
delta,0.043378,0.040203,0.039408,0.039708,0.041134,0.037669,0.046767,0.048735


In [ ]:
table.to_pickle('../model/params/auxiliary.pkl')